# 决策树

In [2]:
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pandas as pd
from math import log

In [3]:
def create_data():
    # 数据集和每个维度的名称
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    datas = pd.DataFrame(datasets, columns=labels)
    print(type(datas))
    
    #X_train, Y_train, X_test, Y_test = train_test_split(datasets, labels, test_size=0.3)
    
    return datas


In [4]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {'label:': self.label, 'feature': self.feature, 'tree': self.tree}

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)
        

In [5]:
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):  # datasets=np.array(train_data) 每个样本的[-1]是最终判断结果(label)
        data_length = len(datasets)    # 样本数量
        label_count = {}
        for i in range(data_length):     # 遍历每个样本
            label = datasets[i][-1]     # 取每个样本的label
            if label not in label_count:     # 以label作为key建立字典
                label_count[label] = 0
            label_count[label] += 1    # 统计每个label的样本个数
            
        # math.log默认以e为底
        ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)   # 样本数量
        feature_sets = {}
        for i in range(data_length):    # 遍历每个样本
            feature = datasets[i][axis]    # 每个样本的第axis个特征（的选项），如第axis=0个特征，可能为青年/中年/老年
            if feature not in feature_sets:
                feature_sets[feature] = []   # 以每个样本的第axis个特征可能的取值为key建立字典 
            feature_sets[feature].append(datasets[i])    # value为该样本（里面可能含有多个样本）
            
        # 将每个样本输入熵计算中   (该特征相同的样本数/样本数量)*该特征相同的所有样本的熵
        cond_ent = sum([(len(p)/data_length)*self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):     # datasets=np.array(train_data)
        count = len(datasets[0]) - 1     # 计算第一个样本的长度（-1是减掉最终判断结果），即特征数量
        ent = self.calc_ent(datasets)  # 计算 熵
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c)) # 遍历输入每个特征，不包括最后的判断值
            best_feature.append((c, c_info_gain))   # 每个特征及其信息增益 ，元组形式保存在列表中
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])# 按元组[-1]位置的值，取最大，返回该元组
        return best_    # 返回的是元组（特征所在位置索引值， 信息增益）

    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
             # iloc索引行，y_train取train_data的最后一列, features按列取每列第一个，即类别。
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:      # ==2 有两类。统计各个类别的个数 是 9  否 6
            return Node(root=True,
                        label=y_train.iloc[0])    # 将第一个的类别输入

        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            # 按数量排序并取最多的那个
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))# 转为特征矩阵输入，计算最大信息增益
        max_feature_name = features[max_feature]    # 根据索引值获得具有最大信息增益的特征名称

        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 5,构建Ag子集
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)

        # 统计该特征下所有类型的值（末尾的.index-> Index(['老年', '中年', '青年'], dtype='object') ）类似set()
        feature_list = train_data[max_feature_name].value_counts().index 
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            # 6, 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        # pprint.pprint(node_tree.tree)
        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [15]:
train_data = create_data()
# print(train_data)
_, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
print(train_data)
feature_list = train_data["年龄"].value_counts().index
print(train_data["年龄"].value_counts().index)
print("y_train = ", y_train)
print("features = ", features)
print(np.array(train_data))
print(y_train.value_counts())
print(y_train.value_counts().sort_values(ascending=False).index[0])


dt = DTree()
tree = dt.fit(train_data)

<class 'pandas.core.frame.DataFrame'>
    年龄 有工作 有自己的房子 信贷情况 类别
0   青年   否      否   一般  否
1   青年   否      否    好  否
2   青年   是      否    好  是
3   青年   是      是   一般  是
4   青年   否      否   一般  否
5   中年   否      否   一般  否
6   中年   否      否    好  否
7   中年   是      是    好  是
8   中年   否      是  非常好  是
9   中年   否      是  非常好  是
10  老年   否      是  非常好  是
11  老年   否      是    好  是
12  老年   是      否    好  是
13  老年   是      否  非常好  是
14  老年   否      否   一般  否
Index(['老年', '中年', '青年'], dtype='object')
y_train =  0     否
1     否
2     是
3     是
4     否
5     否
6     否
7     是
8     是
9     是
10    是
11    是
12    是
13    是
14    否
Name: 类别, dtype: object
features =  Index(['年龄', '有工作', '有自己的房子', '信贷情况'], dtype='object')
[['青年' '否' '否' '一般' '否']
 ['青年' '否' '否' '好' '否']
 ['青年' '是' '否' '好' '是']
 ['青年' '是' '是' '一般' '是']
 ['青年' '否' '否' '一般' '否']
 ['中年' '否' '否' '一般' '否']
 ['中年' '否' '否' '好' '否']
 ['中年' '是' '是' '好' '是']
 ['中年' '否' '是' '非常好' '是']
 ['中年' '否' '是' '非常好' '是']
 ['老年' '否' '是' '非常好' '是']
 ['老年' '

In [35]:
dt.predict(['老年', '否', '否', '一般'])

'否'

# sklearn

In [ ]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz
import graphviz

In [ ]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    # print(data)
    return data[:,:2], data[:,-1]

X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train,)

clf.score(X_test, y_test)

In [ ]:
tree_pic = export_graphviz(clf, out_file="mytree.pdf")
with open('mytree.pdf') as f:
    dot_graph = f.read()

In [ ]:
graphviz.Source(dot_graph)